# 3. DIA

This notebook shows how to run a variety of Data Independent Acquisition (DIA) methods using ViMMS

In [1]:
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('../..')

In [4]:
from pathlib import Path

In [5]:
from vimms.MassSpec import IndependentMassSpectrometer
from vimms.Controller import AIF, SWATH, DiaController
from vimms.Environment import Environment
from vimms.Common import *

### Load the data

In [6]:
data_dir = os.path.abspath(os.path.join(os.getcwd(),'..','..','tests','fixtures'))
dataset = load_obj(os.path.join(data_dir, 'QCB_22May19_1.p'))
ps = load_obj(Path(data_dir,'peak_sampler_mz_rt_int_beerqcb_fragmentation.p'))

### AIF

This example shows how to run an All-Ion Fragmentation (AIF) DIA method on ViMMS. The controller sends an MS1 scan followed by a fragmentation scan which covers the full m/z range.

In [7]:
isolation_width = 1
N = 10
rt_tol = 15
mz_tol = 10
ionisation_mode = POSITIVE

min_mz = 70
max_mz = 1000

min_rt = 0
max_rt = 400

scan_time_dict = {1: 0.12, 2: 0.06}

In [8]:
mass_spec = IndependentMassSpectrometer(ionisation_mode, dataset, ps, scan_duration_dict=scan_time_dict)
controller = AIF(min_mz, max_mz)

In [9]:
# create an environment to run both the mass spec and controller
env = Environment(mass_spec, controller, min_rt, max_rt, progress_bar=True)

# set the log level to WARNING so we don't see too many messages when environment is running
set_log_level_warning()

# run the simulation
env.run()

(400.080s) ms_level=1: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 399.9600000000121/400 [00:56<00:00,  7.09it/s]


Simulated results are saved to the following .mzML file and can be viewed in tools like ToppView or using other mzML file viewers.

In [10]:
set_log_level_debug()
mzml_filename = 'aif_controller.mzML'
out_dir = os.path.join(os.getcwd(), 'results')
env.write_mzML(out_dir, mzml_filename)

2020-09-03 11:05:40.799 | DEBUG    | vimms.Environment:write_mzML:161 - Writing mzML file to C:\Users\Vinny\work\vimms\demo\02. Methods\results\aif_controller.mzML
2020-09-03 11:08:35.693 | DEBUG    | vimms.Environment:write_mzML:164 - mzML file successfully written!


### SWATH

This example shows how to run the SWATH-style method on ViMMS. The controller send an MS1 scan followed by a series of fragmentation scans which cover the full m/z range.

In [11]:
min_mz = 100
max_mz = 1000
width = 100
scan_overlap = 10

ionisation_mode = POSITIVE

scan_time_dict = {1: 0.124, 2: 0.124}

In [12]:
mass_spec = IndependentMassSpectrometer(ionisation_mode, dataset, ps, scan_duration_dict=scan_time_dict)
controller = SWATH(min_mz, max_mz, width, scan_overlap=scan_overlap)

In [13]:
# create an environment to run both the mass spec and controller
env = Environment(mass_spec, controller, min_rt, max_rt, progress_bar=True)

# set the log level to WARNING so we don't see too many messages when environment is running
set_log_level_warning()

# run the simulation
env.run()

(400.024s) ms_level=2: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 399.90000000002055/400 [00:19<00:00, 20.96it/s]


Simulated results are saved to the following .mzML file and can be viewed in tools like ToppView or using other mzML file viewers.

In [14]:
set_log_level_debug()
mzml_filename = 'swath_controller.mzML'
out_dir = os.path.join(os.getcwd(), 'results')
env.write_mzML(out_dir, mzml_filename)

2020-09-03 11:08:55.129 | DEBUG    | vimms.Environment:write_mzML:161 - Writing mzML file to C:\Users\Vinny\work\vimms\demo\02. Methods\results\swath_controller.mzML
c:\users\vinny\.virtualenvs\vimms-rr-f0yt0\lib\site-packages\psims\document.py:393: ReferentialIntegrityWarning: No reference was found for 100000 in Spectrum
  result = super(CallbackBindingPartial, self).__call__(*args, **kwargs)
2020-09-03 11:08:59.675 | DEBUG    | vimms.Environment:write_mzML:164 - mzML file successfully written!


### Nested DIA

This example shows how to run a nested DIA method on ViMMS. This method comes from the following paper: Kaufmann A, Walker S. Nested data independent MS/MS acquisition. *Anal Bioanal Chem*. 2016;408(18):5031-5040. doi:10.1007/s00216-016-9607-8

In [22]:
window_type = 'even' # 'percentile'
kaufmann_design = 'nested'
num_windows=64 # this gets defaulted to this value regardless for this method
scan_overlap=0 # currently fixed at 0 regarless of this setting

ionisation_mode = POSITIVE
scan_time_dict = {1: 0.12, 2: 0.06}

min_rt = 0
max_rt = 400

min_mz = 100
max_mz = 1000

In [23]:
mass_spec = IndependentMassSpectrometer(ionisation_mode, dataset, ps, scan_duration_dict=scan_time_dict)
controller = DiaController(min_mz, max_mz, window_type, kaufmann_design, num_windows, scan_overlap=scan_overlap)

In [25]:
# create an environment to run both the mass spec and controller
env = Environment(mass_spec, controller, min_rt, max_rt, progress_bar=True)

# set the log level to WARNING so we don't see too many messages when environment is running
set_log_level_warning()

# run the simulation
env.run()

(400.020s) ms_level=2: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 399.9600000000134/400 [01:06<00:00,  6.01it/s]


Simulated results are saved to the following .mzML file and can be viewed in tools like ToppView or using other mzML file viewers.

In [26]:
set_log_level_debug()
mzml_filename = 'nest_dia_controller.mzML'
out_dir = os.path.join(os.getcwd(), 'results')
env.write_mzML(out_dir, mzml_filename)

2020-09-03 11:47:11.427 | DEBUG    | vimms.Environment:write_mzML:161 - Writing mzML file to C:\Users\Vinny\work\vimms\demo\02. Methods\results\nest_dia_controller.mzML
2020-09-03 11:47:28.137 | DEBUG    | vimms.Environment:write_mzML:164 - mzML file successfully written!


### Tree DIA

This example shows how to run a tree DIA method on ViMMS. This method comes from the following paper: Kaufmann A, Walker S. Nested data independent MS/MS acquisition. *Anal Bioanal Chem*. 2016;408(18):5031-5040. doi:10.1007/s00216-016-9607-8

In [27]:
window_type = 'even' # 'percentile'
kaufmann_design = 'tree'
num_windows=64 # this gets defaulted to this value regardless for this method
scan_overlap=0 # currently fixed at 0 regarless of this setting

ionisation_mode = POSITIVE
scan_time_dict = {1: 0.12, 2: 0.06}

min_rt = 0
max_rt = 400

min_mz = 100
max_mz = 1000

In [28]:
mass_spec = IndependentMassSpectrometer(ionisation_mode, dataset, ps, scan_duration_dict=scan_time_dict)
controller = DiaController(min_mz, max_mz, window_type, kaufmann_design, num_windows, scan_overlap=scan_overlap)

In [29]:
# create an environment to run both the mass spec and controller
env = Environment(mass_spec, controller, min_rt, max_rt, progress_bar=True)

# set the log level to WARNING so we don't see too many messages when environment is running
set_log_level_warning()

# run the simulation
env.run()

(400.020s) ms_level=2: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 399.9600000000133/400 [01:33<00:00,  4.26it/s]


Simulated results are saved to the following .mzML file and can be viewed in tools like ToppView or using other mzML file viewers.

In [30]:
set_log_level_debug()
mzml_filename = 'tree_dia_controller.mzML'
out_dir = os.path.join(os.getcwd(), 'results')
env.write_mzML(out_dir, mzml_filename)

2020-09-03 11:49:39.475 | DEBUG    | vimms.Environment:write_mzML:161 - Writing mzML file to C:\Users\Vinny\work\vimms\demo\02. Methods\results\tree_dia_controller.mzML
2020-09-03 11:50:04.855 | DEBUG    | vimms.Environment:write_mzML:164 - mzML file successfully written!
